# Customer Life Value Model

Goal: 

Base Model:

Outcomes:

- 

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")
palette = sns.color_palette("Set2")

print("Packages successfully imported!")

Packages successfully imported!
